Will follow [Huggingface's translation tutorial](https://huggingface.co/docs/transformers/tasks/translation) more or less.

In [ ]:
START_EPOCH=0
EPOCHS = 10

In [ ]:
# Stuff for running the same notebook locally and on Google Colab for training
import sys
COLAB_PATH = "/content/drive/MyDrive/Colab Notebooks/diversiformer/"
IN_COLAB = 'google.colab' in sys.modules
fp = COLAB_PATH + "training_data_gender.jsonl" if IN_COLAB else "../data/training_data_gender.jsonl"
if IN_COLAB:
    from google.colab import drive # type: ignore
    drive.mount('/content/drive')
    %pip install transformers datasets sacrebleu sentencepiece carbontracker

In [ ]:
from datasets import load_dataset

data = load_dataset("json", data_files=fp)
data = data["train"].train_test_split(test_size=0.2, shuffle=False)
data["train"][:3]

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

In [ ]:
def preprocess_function(data):
    inputs = [f"""Ersetze "{a}" durch "{b}": {x}""" for a, b, x in zip(data["a"], data["b"], data["x"])]
    print(inputs)
    targets = data["y"]
    model_inputs = tokenizer(inputs)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_data = data.map(preprocess_function, batched=True)

In [ ]:
from transformers import TFT5ForConditionalGeneration

try:
    # load model
    model = TFT5ForConditionalGeneration.from_pretrained((COLAB_PATH if IN_COLAB else "../data/") + f"checkpoint_{START_EPOCH}_epochs")
except:
    print("WARNING: Could not load local model.")
    model = TFT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
examples = """
Ersetze "Autofahrer" durch "Mensch, der Auto fährt": Heute morgen im Stau haben mich die Autofahrer wieder sehr aufgeregt.
Ersetze "Behinderter" durch  "Mensch mit Behinderungen": Behinderte mit entsprechenden Ausweisen bekommen ermäßigten Eintritt.
Ersetze "Student" durch "studierende Person" bzw. "Studierende": Viele faule Studenten studieren gar nicht wirklich.
Ersetze "Student" durch "studierende Person" bzw. "Studierende": Maria ist kein Student.
Ersetze "Lehrer" durch "Lehrerin oder Lehrer" bzw. "Kollegium": Die Lehrer machen morgen einen Ausflug.
Ersetze "Lehrer" durch "Lehrerin oder Lehrer" bzw. "Kollegium": Ein promovierter Mathelehrer ist noch nie im Unterricht eingeschlafen.
Ersetze "Polizist" durch "Polizistin oder Polizist": Die Polizisten machen oft Überstunden.
Ersetze "Gaul" durch "Stute oder Gaul": Einem geschenkten Gaul schaut man nicht ins Maul.
""".strip().split("\n")

In [ ]:
if IN_COLAB:
    import json
    from carbontracker.tracker import CarbonTracker
    from transformers import pipeline

    tracker = CarbonTracker(epochs=EPOCHS, verbose=2)
    try:
        with open(COLAB_PATH + "example_predictions.json") as f:
            example_eval = json.load(f)
    except:
        example_eval = []
    example_eval = []
    for epoch in range(1, EPOCHS + 1):
        print(f"Epoch {START_EPOCH + epoch}")
        tracker.epoch_start()
        model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)
        tracker.epoch_end()
        generator = pipeline(
            task="text2text-generation", model=model, tokenizer=tokenizer
        )
        example_eval.append(
            [
                dict(
                    epoch=START_EPOCH + epoch,
                    prompt=prompt,
                    response=generator(prompt)[0]["generated_text"],
                )
                for prompt in examples
            ]
        )
        with open(COLAB_PATH + "example_predictions.json", "w") as f:
            json.dump(example_eval, f, indent=2, ensure_ascii=False)
    tracker.stop()


In [ ]:
if IN_COLAB:
    model.save_pretrained(COLAB_PATH + f"checkpoint_{START_EPOCH+EPOCHS}_epochs")
    model.save(COLAB_PATH + f"tf_checkpoint_{START_EPOCH+EPOCHS}_epochs")

In [ ]:
from transformers import pipeline

generator = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer)

In [ ]:
entries = list(zip(data["test"]["x"], data["test"]["a"], data["test"]["b"], data["test"]["y"]))

In [ ]:
for x, a, b, y in entries[:5]:
    prompt = f"""Ersetze "{a}" durch "{b}": {x}"""
    print(prompt)
    prediction = generator(prompt)[0]["generated_text"]
    print(prediction)
    print()